# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [87]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import json

# Import API key
from api_keys import geoapify_key



In [88]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head(15)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,provideniya,64.3833,-173.3000,28.85,93,100,20.07,RU,1667831083
1,1,port hardy,50.6996,-127.4199,32.18,64,20,19.57,CA,1667831083
2,2,saint anthony,45.0205,-93.2180,28.45,72,0,10.36,US,1667830497
3,3,dikson,73.5069,80.5464,7.74,91,100,11.36,RU,1667831084
4,4,egvekinot,66.3167,-179.1667,24.33,96,100,10.00,RU,1667831084
5,5,hobart,-42.8794,147.3294,55.06,92,53,4.61,AU,1667830867
6,6,castro,-24.7911,-50.0119,63.91,50,0,5.14,BR,1667831084
7,7,rikitea,-23.1203,-134.9692,72.84,86,100,22.10,PF,1667831085
8,8,cidreira,-30.1811,-50.2056,64.96,74,6,11.97,BR,1667831085
9,9,port elizabeth,-33.9180,25.5701,74.10,75,0,19.57,ZA,1667831085


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [89]:
#capture --no-display

# Configure the map plot
coordinates = {
    'latitude': city_data_df['Lat'],
    'longitude': city_data_df['Lng'],
    'City': city_data_df["City"],
    'Population': city_data_df['Humidity']
    
}
coordinates_df = pd.DataFrame(coordinates)

map_plot_1 = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = 'Population',
    color = "City"
    )

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Population)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [90]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_values_df = city_data_df

filtered_values_df = filtered_values_df.loc[filtered_values_df['Wind Speed'] < 20] 
                                            
filtered_values_df = filtered_values_df.loc[filtered_values_df['Cloudiness'] < 10]  

filtered_values_df = filtered_values_df.loc[filtered_values_df['Max Temp'] < 75] 
     
# Drop any rows with null values
filtered_values_df = filtered_values_df.dropna()


# Display sample data

filtered_values_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,saint anthony,45.0205,-93.2180,28.45,72,0,10.36,US,1667830497
6,6,castro,-24.7911,-50.0119,63.91,50,0,5.14,BR,1667831084
8,8,cidreira,-30.1811,-50.2056,64.96,74,6,11.97,BR,1667831085
9,9,port elizabeth,-33.9180,25.5701,74.10,75,0,19.57,ZA,1667831085
20,20,digoin,46.4812,3.9795,66.29,45,0,5.75,FR,1667831087
...,...,...,...,...,...,...,...,...,...,...
515,515,talcahuano,-36.7167,-73.1167,56.50,73,5,7.81,CL,1667831216
524,524,along,28.1667,94.7667,62.55,88,0,1.12,IN,1667831218
531,531,jinan,36.6683,116.9972,53.02,29,0,4.79,CN,1667831218
532,532,portage,41.1667,-81.1998,55.83,57,0,10.36,US,1667831220


### Step 3: Create a new DataFrame called `hotel_df`.

In [91]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_values_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df = filtered_values_df[['City','Country','Lat','Lng','Humidity']].copy() # added this 

hotel_df['Hotel Name'] = ''
tmp_hotel_df = hotel_df # This is temporary to limit the results.
# Display sample data
hotel_df = hotel_df.head()
hotel_df

# used this to limit to only 5

,City,Country,Lat,Lng,Humidity,Hotel Name
2,saint anthony,US,45.0205,-93.2180,72,
6,castro,BR,-24.7911,-50.0119,50,
8,cidreira,BR,-30.1811,-50.2056,74,
9,port elizabeth,ZA,-33.9180,25.5701,75,
20,digoin,FR,46.4812,3.9795,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [92]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit =20
params = {
    "categories":categories,
    "apiKey": geoapify_key,
    "limit":limit,
    'format': 'json',
    'filter': '',
    'bias': '',
}
# https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:-75.5053739,40.2301628,10000&bias=proximity:-75.5053739,40.2301628&limit=1&apiKey=YOUR_API_KEY
# https://api.geoapify.com/v2/places?categories=airport&filter=circle:-75.5053739,40.2301628,10000&bias=proximity:-75.5053739,40.2301628&limit=1&apiKey=YOUR_API_KEY

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    name_address
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


C:\Users\19179\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


saint anthony - nearest hotel: Delta Hotels by Marriott
castro - nearest hotel: Hotel Central Palace
cidreira - nearest hotel: Hotel Castelo
port elizabeth - nearest hotel: Waterford Hotel
digoin - nearest hotel: Entre Mer et Montagne


,City,Country,Lat,Lng,Humidity,Hotel Name
2,saint anthony,US,45.0205,-93.2180,72,Delta Hotels by Marriott
6,castro,BR,-24.7911,-50.0119,50,Hotel Central Palace
8,cidreira,BR,-30.1811,-50.2056,74,Hotel Castelo
9,port elizabeth,ZA,-33.9180,25.5701,75,Waterford Hotel
20,digoin,FR,46.4812,3.9795,45,Entre Mer et Montagne


In [93]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,saint anthony,US,45.0205,-93.2180,72,Delta Hotels by Marriott
6,castro,BR,-24.7911,-50.0119,50,Hotel Central Palace
8,cidreira,BR,-30.1811,-50.2056,74,Hotel Castelo
9,port elizabeth,ZA,-33.9180,25.5701,75,Waterford Hotel
20,digoin,FR,46.4812,3.9795,45,Entre Mer et Montagne


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [113]:
%%capture --no-display
hotel_df = pd.DataFrame(hotel_df)
# Configure the map plot
coordinates = {
    'latitude': hotel_df['Lat'],
    'longitude': hotel_df['Lng'],
    'City': hotel_df["City"],
    'Hotel Name': hotel_df["Hotel Name"]
       }
hotel_df_update = pd.DataFrame(coordinates)

map_plot_1 = hotel_df_update.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 500,
    size = 'Population',
    color = "City",
    hover_cols=['Hotel Name']
    )



# Display the map
map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (City,Hotel Name)

,City,Country,Lat,Lng,Humidity,Hotel Name
2,saint anthony,US,45.0205,-93.2180,72,Delta Hotels by Marriott
6,castro,BR,-24.7911,-50.0119,50,Hotel Central Palace
8,cidreira,BR,-30.1811,-50.2056,74,Hotel Castelo
9,port elizabeth,ZA,-33.9180,25.5701,75,Waterford Hotel
20,digoin,FR,46.4812,3.9795,45,Entre Mer et Montagne
